# 04. Test MLflow Serving (Docker)

Ce notebook teste l'API exposée par le conteneur Docker.
Assurez-vous que le conteneur tourne sur le port 5000.
Commande: `docker run -p 5000:5000 -v $(pwd)/models/final_model:/model -e MLFLOW_MODEL_URI=/model credit-scoring-serving`

In [ ]:
import requests
import pandas as pd
import json
import numpy as np

In [ ]:
# Chargement d'un sample de données (V1 ou V2, peu importe tant que le format colonnes est bon)
# Idéalement prendre le même dataset que celui du gagnant
X_test = pd.read_pickle('../data/processed/X_prepared_v1.pkl')
# Clean colonnes comme pour l'entrainement
X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

sample = X_test.sample(5)
print("Sample shape:", sample.shape)

In [ ]:
# Endpoint par defaut MLflow serve
url = "http://127.0.0.1:5000/invocations"

# Format 'dataframe_split' est souvent requis par MLflow standard
data_json = sample.to_dict(orient='split')
# data_json contient 'index', 'columns', 'data'

headers = {'Content-Type': 'application/json'}

try:
    response = requests.post(url, json={"dataframe_split": data_json}, headers=headers)
    print("Status Code:", response.status_code)
    
    if response.status_code == 200:
        print("Predictions:", response.json())
    else:
        print("Erreur:", response.text)
except Exception as e:
    print("Erreur connexion:", e)
    print("Vérifiez que le docker tourne bien.")